# Website A/B Testing - Lab done by `Eugene Maina`

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [16]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [44]:
#Your code here
'''
H_0 = control group has more have a higher click to views rate than the experimental
H_1 = experimental group have a higher click to views rate than control

'''
df['count'] = 1

# Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

# Calculate the number of views and clicks for each group
control_views = control['view'].sum()
control_clicks = control['click'].sum()
experimental_views = experiment['view'].sum()
experimental_clicks = experiment['click'].sum()

# Calculate the click-through rate for each group
control_ctr = control_clicks / control_views
experimental_ctr = experimental_clicks / experimental_views

print(f"Control Click-Through Rate: {control_ctr}")
print(f"Experimental Click-Through Rate: {experimental_ctr}")

Control Click-Through Rate: 0.2797118847539016
Experimental Click-Through Rate: 0.3097463284379172


In [45]:
from scipy.stats import ttest_ind

alpha = 0.05

# Conducting the t-test
t_stat, p_val = ttest_ind(control['click'], experiment['click'])

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

if p_val <= alpha:
    print('Reject Null Hypothesis')
else:
    print('Fail to reject Null Hypothesis')

T-statistic: -2.6195696844542207
P-value: 0.008825098914958288
Reject Null Hypothesis


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [47]:
#Your code here
experiment_expected_number_clicks = experimental_views * control_ctr
experiment_expected_number_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [48]:
#Your code here
standard_deviation = np.sqrt(experiment_expected_number_clicks * (1 - control_ctr))
standard_deviation

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [52]:
#Your code here


z_score = (experimental_clicks - experiment_expected_number_clicks) / standard_deviation
z_score

p_value = stats.norm.sf(z_score)

print(f'The p-value is {p_value}')

if p_value <= alpha:
    print('Reject Null Hypothesis')
else:
    print('Fail to reject Null Hypothesis')



The p-value is 0.00012486528006951198
Reject Null Hypothesis


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p values are not the same however both conclude that the Null hypothesis should be rejected**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.